In [37]:
import numpy as np
import pandas as pd
from surprise import Reader, Dataset
from surprise import SVD
from surprise import SVD, NMF, KNNWithMeans, CoClustering
import surprise
from surprise.model_selection import train_test_split


(il faut pip install "surprise")

In [38]:
data = pd.read_csv('dataset_movie_1M.csv')[['user_id','book_id','rating']]

In [39]:
print("Le dataset contient ", data.shape[0], " notes.")

data.head()

Le dataset contient  1000209  notes.


,user_id,book_id,rating
0,2599,1,4
1,18,1,4
2,4089,1,5
3,3626,1,4
4,2873,1,5


In [40]:
reader = Reader()
data = Dataset.load_from_df(data, reader)

trainset, testset = train_test_split(data, test_size=.25)

In [46]:
#algo = SVD(n_factors=10, n_epochs=10,lr_all=.01, reg_all=.01)
algo= NMF(10)
#algo=KNNWithMeans(epochs=10)
#algo=CoClustering()

In [47]:
#results = surprise.model_selection.validation.cross_validate(SVD, trainset, measures=['MSE'], cv=3, verbose=True)

## precison et recall

### copier coller de la fonction dans la FAQ

In [48]:
#https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k

In [49]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


def F_score(pr,rc):
    return 2*(pr*rc)/(rc+pr)

### K-fold (ici 5-fold)

In [ ]:
# si tu veut le faire plus proprement avec 5-fold (un peu comme CV)
kf = KFold(n_splits=5)
ix_fold = 1

res_precision = []
res_recall= []
f_score=[]

for trainset, testset in kf.split(data):
    print('='*50)
    print ('FOLD # ' + str(ix_fold))
    print('='*50)
    
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

    # Precision and recall can then be averaged over all users
    avg_precision = 1.0*sum(prec for prec in precisions.values()) / len(precisions)
    avg_recall = 1.0*sum(rec for rec in recalls.values()) / len(recalls)
    f=F_score(avg_precision,avg_recall)
    
    print (avg_precision)
    print (avg_recall)
    print(f)
    
    res_precision.append(avg_precision)
    res_recall.append(avg_recall)
    f_score.append(f)
    ix_fold += 1
    
    
print('='*50)
print ('SUMMARY')
print('='*50)
print('avg precision accross all folds: ' + str(np.mean(res_precision)))
print('avg recall accross all folds: ' + str(np.mean(res_recall)))
print('avg F_score accross all folds: ' + str(np.mean(f_score)))


FOLD # 1
0.9088863643535652
0.22537891490852163
0.36119208851464085
FOLD # 2


In [ ]:
avg precision accross all folds: 0.8791600385248808
avg recall accross all folds: 0.35939375698587206
avg F_score accross all folds: 0.5102141672914685
